### EVALUACIÓN API NATALIA BARQUÍN

----------------

In [241]:
import requests
import pandas as pd
import numpy as np
import sidetable
from geopy.geocoders import Nominatim
import mysql.connector

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import os
from dotenv import load_dotenv
load_dotenv()

True

1. Utilizando la API extraed toda la información que podáis de ella.

In [242]:
def extraccion_df_api():

    """
    Crea una lista con los 3 países que queremos estudiar y realiza la llamada a la API y extracción de datos de la misma para cada uno de los países.
    Después crea un dataframe con los datos de cada país y los unifica en un único dataframe.

    Args:
        No recibe.
        
    Returns:
        df: dataframe unido con los datos de los 3 países.
    """

    lista_paises = ['United States', 'Canada' , 'Argentina']

    df_unido = pd.DataFrame()

    for pais in lista_paises:

        url = f'http://universities.hipolabs.com/search?country={pais}'

        response = requests.get(url=url)

        response.status_code

        response.reason
        
        df = pd.json_normalize(response.json())

        df_unido = pd.concat([df_unido, df], ignore_index = True)
    
    return df_unido

In [243]:
df_universidades = extraccion_df_api()
df_universidades.head()

,web_pages,alpha_two_code,state-province,name,domains,country
0,[http://www.marywood.edu],US,None,Marywood University,[marywood.edu],United States
1,[http://www.lindenwood.edu/],US,None,Lindenwood University,[lindenwood.edu],United States
2,[https://sullivan.edu/],US,None,Sullivan University,[sullivan.edu],United States
3,[https://www.fscj.edu/],US,None,Florida State College at Jacksonville,[fscj.edu],United States
4,[https://www.xavier.edu/],US,None,Xavier University,[xavier.edu],United States


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

- Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.

- La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

In [244]:
def homogeneizar_columnas(df):

    """Reemplaza los guiones de los nombres de las columnas del dataframe por barra baja.

    Args:
        df: dataframe original.

    Returns:
        df: dataframe modificado.
    """
    
    return df.rename(columns = {col: col.replace('-', '_') for col in df.columns}, inplace = True)

In [245]:
homogeneizar_columnas(df_universidades)

In [246]:
def eliminar_columnas(df, col):

    """Elimina una columna de un dataframe.

    Args:
        df: dataframe original.
        col: columna a eliminar.

    Returns:
        df: dataframe sin la columna eliminada.
    """

    return df.drop(col, axis = 1, inplace = True)

In [247]:
eliminar_columnas(df_universidades, 'domains')

In [248]:
df_universidades.head(1)

,web_pages,alpha_two_code,state_province,name,country
0,[http://www.marywood.edu],US,None,Marywood University,United States


3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. 

In [249]:
df_universidades[df_universidades['name'] == 'Cégep de Saint-Jérôme']

,web_pages,alpha_two_code,state_province,name,country
4536,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA,Quebec,Cégep de Saint-Jérôme,Canada
4686,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA,Quebec,Cégep de Saint-Jérôme,Canada


In [250]:
def aplicar_explode(df, col):

    """Reemplaza los guiones de los nombres de las columnas del dataframe por barra baja.

    Args:
        df: dataframe original.

    Returns:
        df: dataframe modificado.
    """
    
    return df.explode(col, ignore_index= True)

In [251]:
df_universidades = aplicar_explode(df_universidades, 'web_pages')

In [252]:
df_universidades

,web_pages,alpha_two_code,state_province,name,country
0,http://www.marywood.edu,US,None,Marywood University,United States
1,http://www.lindenwood.edu/,US,None,Lindenwood University,United States
2,https://sullivan.edu/,US,None,Sullivan University,United States
3,https://www.fscj.edu/,US,None,Florida State College at Jacksonville,United States
4,https://www.xavier.edu/,US,None,Xavier University,United States
...,...,...,...,...,...
5029,http://www.untref.edu.ar/,AR,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina
5030,http://www.utdt.edu/,AR,Ciudad Autónoma de Buenos Aires,Universidad Torcuato di Tella,Argentina
5031,http://www.utn.edu.ar/,AR,Ciudad Autónoma de Buenos Aires,Universidad Tecnológica Nacional,Argentina
5032,http://www.vaneduc.edu.ar/uai/,AR,Ciudad Autónoma de Buenos Aires,Universidad Abierta Interamericana,Argentina


4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [253]:
def eliminar_duplicados(df, col):
    
    if df.duplicated([col]).sum() > 0:
        print(f'Tenemos {df.duplicated([col]).sum()} duplicados en la columna "{col}", los eliminaremos.')
        return df.drop_duplicates(subset = col, inplace = True, ignore_index = True)
    
    else:
        print(f'No tenemos duplicados en la columna {col}.')

In [254]:
eliminar_duplicados(df_universidades, 'name')

Tenemos 2543 duplicados en la columna "name", los eliminaremos.


5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [255]:
df_universidades.stb.missing()

,missing,total,percent
state_province,2204,2491,88.478523
web_pages,0,2491,0.000000
alpha_two_code,0,2491,0.000000
name,0,2491,0.000000
country,0,2491,0.000000


In [256]:
def convertir_nulos(valor):
    """convierte los valores None a np.nan

    Args:
        valor (str): _description_

    Returns:
        _type_: _description_
    """

    if valor == None:
        return np.nan
    else:
        return valor

In [257]:
df_universidades['state_province'] = (df_universidades['state_province'].apply(convertir_nulos))

In [258]:
df_universidades.head()

,web_pages,alpha_two_code,state_province,name,country
0,http://www.marywood.edu,US,NaN,Marywood University,United States
1,http://www.lindenwood.edu/,US,NaN,Lindenwood University,United States
2,https://sullivan.edu/,US,NaN,Sullivan University,United States
3,https://www.fscj.edu/,US,NaN,Florida State College at Jacksonville,United States
4,https://www.xavier.edu/,US,NaN,Xavier University,United States


In [259]:
df_universidades.stb.missing()

,missing,total,percent
state_province,2204,2491,88.478523
web_pages,0,2491,0.000000
alpha_two_code,0,2491,0.000000
name,0,2491,0.000000
country,0,2491,0.000000


6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [260]:
df_universidades['state_province'].replace(np.nan, 'Unknow', inplace = True)

#revisar libreria klearn que tambien se puede hacer asi

In [261]:
df_universidades.stb.missing()

,missing,total,percent
web_pages,0,2491,0.0
alpha_two_code,0,2491,0.0
state_province,0,2491,0.0
name,0,2491,0.0
country,0,2491,0.0


In [262]:
df_universidades

,web_pages,alpha_two_code,state_province,name,country
0,http://www.marywood.edu,US,Unknow,Marywood University,United States
1,http://www.lindenwood.edu/,US,Unknow,Lindenwood University,United States
2,https://sullivan.edu/,US,Unknow,Sullivan University,United States
3,https://www.fscj.edu/,US,Unknow,Florida State College at Jacksonville,United States
4,https://www.xavier.edu/,US,Unknow,Xavier University,United States
...,...,...,...,...,...
2486,http://www.untref.edu.ar/,AR,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina
2487,http://www.utdt.edu/,AR,Ciudad Autónoma de Buenos Aires,Universidad Torcuato di Tella,Argentina
2488,http://www.utn.edu.ar/,AR,Ciudad Autónoma de Buenos Aires,Universidad Tecnológica Nacional,Argentina
2489,http://www.vaneduc.edu.ar/uai/,AR,Ciudad Autónoma de Buenos Aires,Universidad Abierta Interamericana,Argentina


7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso. Para desarrollar este ejercicio deberéis: 

- Sacar los valores únicos de la columna state_province.

In [263]:
df_universidades['state_province'].unique()

array(['Unknow', 'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA',
       'TX', 'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA',
       'Illinois', 'New Hampshire', 'North Carolina', 'South Carolina',
       'Virginia', 'Washington', 'Missouri', 'California', 'NY', 'ND',
       'MI', 'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos:

In [264]:
dicc_estados = {
    'NV' : 'Nevada',
    'TX' : 'Texas',
    'IN' : 'Indianapolis',
    'CA' : 'California',
    'VA' : 'Virginia',
    'NY' : 'New York',
    'MI' : 'Michigan', 
    'GA' : 'Georgia', 
    'ND' : 'North Dakota', 
    'New York, NY' : 'New York', 
    'Ciudad Autónoma de Buenos Aires' : 'Buenos Aires', 
}

In [265]:
df_universidades["state_province"] = df_universidades['state_province'].apply(lambda estado: dicc_estados.get(estado, estado))

In [266]:
df_universidades['state_province'].unique()

array(['Unknow', 'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa',
       'Virginia', 'Indiana', 'Colorado', 'Ohio', 'Indianapolis',
       'New York', 'California', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Washington', 'Missouri',
       'North Dakota', 'Michigan', 'Florida', 'Georgia', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Santiago Del Estero', 'Misiones', 'Catamarca',
       'Formosa', 'Jujuy', 'La Rioja', 'La Pampa', 'San Juan', 'San Luis',
       'Tucumán'], dtype=object)

- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [267]:
lista_estados = df_universidades['state_province'].unique().tolist()

- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [268]:
def sacar_latitud_longitud(lista):

    lista_latitud = []
    lista_longitud = []

    for estado in lista:

        if estado != 'Unknow':
            geo = Nominatim(user_agent = 'Natalia')
            localizacion = geo.geocode(estado)
            lista_latitud.append(localizacion.latitude)
            lista_longitud.append(localizacion.longitude)
    
        else:
            lista_latitud.append(np.nan)
            lista_longitud.append(np.nan)


    diccionario = {
        'state_province' : lista,
        'latitude' : lista_latitud,
        'longitude' : lista_longitud
        }

    return pd.DataFrame(diccionario)


In [269]:
#df_coordenadas = sacar_latitud_longitud(lista_estados)
#df_coordenadas.to_csv('../data/coordenadas.csv')

In [270]:
df_coordenadas = pd.read_csv('../data/coordenadas.csv', index_col = 0)

- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [271]:
df_unido = df_universidades.merge(df_coordenadas, on = 'state_province')
df_unido

,web_pages,alpha_two_code,state_province,name,country,latitude,longitude
0,http://www.marywood.edu,US,Unknow,Marywood University,United States,NaN,NaN
1,http://www.lindenwood.edu/,US,Unknow,Lindenwood University,United States,NaN,NaN
2,https://sullivan.edu/,US,Unknow,Sullivan University,United States,NaN,NaN
3,https://www.fscj.edu/,US,Unknow,Florida State College at Jacksonville,United States,NaN,NaN
4,https://www.xavier.edu/,US,Unknow,Xavier University,United States,NaN,NaN
...,...,...,...,...,...,...,...
2486,http://www.unlar.edu.ar/,AR,La Rioja,Universidad Nacional de La Rioja,Argentina,42.281464,-2.482805
2487,http://www.unlpam.edu.ar/,AR,La Pampa,Universidad Nacional de La Pampa,Argentina,-37.231464,-65.397295
2488,http://www.unsj.edu.ar/,AR,San Juan,Universidad Nacional de San Juan,Argentina,18.465299,-66.116666
2489,http://www.unsl.edu.ar/,AR,San Luis,Universidad Nacional de San Luis,Argentina,38.628028,-90.191015


8. Crea una BBDD en mysql que contenga las siguientes tablas:

In [272]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
                                host="localhost",
                                user="root",
                                password= os.getenv('password'))
                                #auth_plugin = 'mysql_native_password') # esta línea no es necesaria solo lo tendréis que incluir si os sale el siguiente error: Authentication plugin 'caching_sha2_password' is not supported
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
        mydb.close()

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        mydb.close()

In [273]:
crear_bbdd('Universidades')

Conexión realizada con éxito
CMySQLCursor: CREATE DATABASE IF NOT EXISTS Universida..


In [274]:
def crear_tabla(nombre_bbdd, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(
                                user='root',
                                password = os.getenv('password'),
                                host='127.0.0.1',
                                database=f"{nombre_bbdd}")  
                                #auth_plugin = 'mysql_native_password'
    print("Conexión realizada con éxito")

    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        #cnx.commit() 
        cnx.close()

    # en caso de que no podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        cnx.close()

In [275]:
tabla_paises = '''
CREATE TABLE IF NOT EXISTS `Universidades`.`paises` (
    `id_estado` INT NOT NULL AUTO_INCREMENT,
    `nombre_pais` VARCHAR(45) NOT NULL,
    `nombre_provincia` VARCHAR(45),
    `latitud` FLOAT,
    `longitud`FLOAT,
    PRIMARY KEY (`id_estado`));
'''

tabla_universidades = '''
CREATE TABLE IF NOT EXISTS `Universidades`.`universidades` (
    `id_universidades` INT NOT NULL AUTO_INCREMENT,
    `nombre_universidad` VARCHAR(100) NOT NULL,
    `pagina_web` VARCHAR(100),
    `paises_id_estado` INT NOT NULL,
    PRIMARY KEY (`id_universidades`),
    CONSTRAINT `fk_universidades_paises`
        FOREIGN KEY (`paises_id_estado`)
        REFERENCES `Universidades`.`paises` (`id_estado`));
ENGINE = InnoDB;
'''

In [276]:
crear_tabla('Universidades', tabla_paises)

Conexión realizada con éxito


In [277]:
crear_tabla('Universidades', tabla_universidades)

Conexión realizada con éxito


9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs.

*Se han ido creando ejercicio por ejercicio.*

10. BONUS

- Introduce los datos en la BBDD de SQL.

- Crea una clase con todo el código generado en esta evaluación.

In [278]:
df_unido.head(1)

,web_pages,alpha_two_code,state_province,name,country,latitude,longitude
0,http://www.marywood.edu,US,Unknow,Marywood University,United States,NaN,NaN


In [279]:
def insertar_tabla(nombre_bbdd, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(
                                user='root',
                                password = os.getenv('password'),
                                host='127.0.0.1',
                                database=f"{nombre_bbdd}")  
                                #auth_plugin = 'mysql_native_password'


    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
        cnx.close()

    # en caso de que no podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        cnx.close()

In [280]:
for indice, fila in df_unido.iterrows():

        latitud = fila['latitude']
        longitud = fila['longitude']

        if pd.isna(latitud):
                latitud = 'NULL'

        if pd.isna(longitud):
                longitud = 'NULL'

        id_estado = sacar_id_estado('Universidades', fila['state_province'],fila['country'])

        if type(id_estado) is not int:

                insertar_paises = f"""
                        INSERT INTO `Universidades`.`paises` (nombre_pais, nombre_provincia, latitud, longitud) 
                        VALUES ( "{fila['country']}", "{fila['state_province']}", {latitud}, {longitud});
                        """
                insertar_tabla('Universidades', insertar_paises)
        else:
                pass

SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
SELECT id_estado FROM paises W

In [282]:
def sacar_id_estado(nombre_bbdd, estado, pais):
    
    cnx = mysql.connector.connect(
                                user='root',
                                password = os.getenv('password'),
                                host='127.0.0.1',
                                database=f"{nombre_bbdd}")  
    mycursor = cnx.cursor()

    query_sacar_id = f"SELECT id_estado FROM paises WHERE nombre_provincia = '{estado}' and nombre_pais = '{pais}'"
    
    # puede que el id de la ciudad que estemos intentando insertar no este en nuestra BBDD. Por lo que vamos a hacer un try except para que no nos falle el código
    try: 
        mycursor.execute(query_sacar_id)
        id_ = mycursor.fetchall()[0][0]
        
        return id_
    
    except: 
        return "Lo siento, no tenemos ese estado en la BBDD y por lo tanto no te podemos dar su id. "

In [86]:
df_unido.head(1)

,web_pages,alpha_two_code,state_province,name,country,latitude,longitude
0,http://www.marywood.edu,US,Unknow,Marywood University,United States,NaN,NaN


In [107]:
for indice, fila in df_unido.iterrows():

        id_estado = sacar_id_estado('Universidades', fila['state_province'], fila['country'])
        print(id_estado)
        insertar_universidades = f"""
                INSERT INTO `Universidades`.`universidades` (nombre_universidad, pagina_web, paises_id_estado) 
                VALUES ( "{fila['name']}", "{fila['web_pages']}", {id_estado});
                """
        
        insertar_tabla('Universidades', insertar_universidades)

SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_estado FROM paises WHERE nombre_provincia = 'Unknow' and nombre_pais = 'United States'
1
SELECT id_